# Assignment 2 - part-of-speech prediction from limited context

In this assignment, you will train classifiers that attempt, within a window of five words, to make a binary prediction about whether the third word belongs to a given part of speech (noun, verb, adjective, adverb), but using very limited information -- that is, the last two letters of the first, second, fourth, and fifth word of the sequence, and no information whatsoever directly from the third word itself.  You will strip out all punctuation (using the NLTK `WordPunctTokenizer`), lowercase, and remove stop words (using the NLTK English stop words list).

In other words, you will predict over samples that have two classes, P and not-P, where P is the selected part of speech to classify.  For example, from the sentence, "The quick brown fox jumped over the lazy dog.", we can select the following 5-word windows without stop words, "brown fox jumped lazy dog" and "quick brown fox jumped lazy".  If we select verbs as the part-of-speech we are classifying over, we get the instances <(wn,ox,zy,og),1>, since "jumped" is a verb, but <(ck,wn,ed,zy),0> because "fox" in that context is not.

This means that you will need to take into account the position of the last-two-letter feature:  "zy" as the fourth word's last two letters is different from "zy" as the fifth word's last two letters.  They are two features, say, `zy_4` and `zy_5`.

This will likely not actually work.  But it might!

You will create training and testing samples according to this procedure, and you will build a data structure that can be fed to a support vector machine (SVM) classifier.  You will train the classifier on the training data and evaluate it on the testing data. 

The work will be done in a .py module file in the same folder as this notebook.  **No modifications to this notebook will be graded.** We will run your module using this notebook or one we modify that you won't see in order to test your code.

The file you must create and add to the github repo is `mycode.py`, which will be imported here.  You can create your own notebooks or scripts to test it.  You can put any number of your own helper functions and also put optional parameters on any of the python functions mentioned here. You should also create a Markdown file, `notes.md`, to keep any **concise** notes and remarks about the assignment.  The code must run on mltgpu.

**This assignment is due Monday, 2022 March 7, at 23:59. There are 33 points and 5 bonus points.**

In [1]:
import mycode as mc

## Part 0 - preparation (2 points)

Fork this repository and create and add `mycode.py` and `notes.md`. 

## Part 1 - obtaining the text (3 points)

You will randomly select the given number of lines from the gzipped file we give you (so you will have to figure out how to access gzipped text files).  Explain how you implemented the random selection in `notes.md`. When we run it, it should give a new sample every time.

In [3]:

sampled_lines = mc.sample_lines("/scratch/UN-english.txt.gz", lines=100000)

print(len(sampled_lines))
sampled_lines[40000:40010]

100000


[b'PMIF provides cover against loan defaulting, insuring loans over their full term on a commercially viable basis.\n',
 b'On Thursday, 20 February, a conference link-up was arranged with representatives of the United Nations Development Programme, the secretariat of the Permanent Forum in DESA and the World Bank office in New York.\n',
 b'The provisions of draft article 14 were intended to fill in the gaps where the intention of the parties remained unclear.\n',
 b'Report of the Special Rapporteur on adequate housing as a component\n',
 b'For the "second wave" of countries behind the potential graduation cases (i.e. countries demonstrating some structural progress, but not yet foreseeing graduation in the near future), country-specific packages of measures could be formulated in the light of an analysis of structural disadvantages as measured under the LDC criteria.\n',
 b'Its outbreak and spread pose a threat to public health undertakings.\n',
 b'The Council elected Jordan for a term

## Part 2 - creating the samples (7 points)

From the sampled lines, you will then randomly create the five-word samples.

You will tokenize the sentences and apply POS-tagging to them -- you need to do this before you create the samples, since POS-tagging needs context. You will then remove stop words and punctuation and lowercase the remainder.  Next, you will randomly, over the entire set of sentences, choose samples of five words in sequence, up to a certain limit.  You find the last two characters of the first, second, fourth, and fifth words, and create the type of structure specified up in the introduction to this assignment for each sample. The exact representation is up to you.

In [4]:
processed_sentences = mc.process_sentences(sampled_lines)
processed_sentences[40000:40010]

[            word  POS
 0           pmif  NNP
 1       provides  VBZ
 2          cover   NN
 3           loan   NN
 4     defaulting   NN
 5       insuring  VBG
 6          loans  NNS
 7           full   JJ
 8           term   NN
 9   commercially   RB
 10        viable   JJ
 11         basis   NN,
                word  POS
 0          thursday  NNP
 1          february  NNP
 2        conference   NN
 3              link   NN
 4          arranged  VBN
 5   representatives  NNS
 6            united  NNP
 7           nations  NNP
 8       development  NNP
 9         programme  NNP
 10      secretariat   NN
 11        permanent  NNP
 12            forum  NNP
 13             desa  NNP
 14            world  NNP
 15             bank  NNP
 16           office   NN
 17              new  NNP
 18             york  NNP,
          word  POS
 0  provisions  NNS
 1       draft   NN
 2     article   NN
 3    intended  VBN
 4        fill   VB
 5        gaps  NNS
 6   intention   NN
 7     parties  NNS

In [5]:
all_samples = mc.create_samples(processed_sentences, samples=50000)

print(all_samples[25000:25010])

[            word POS
1           like  VB
2           take  VB
3    opportunity  NN
4          thank  VB
5  participation  NN,             word  POS
1         growth   NN
2        however   RB
3         united  NNP
4        nations  NNP
5  organizations  NNS,            word  POS
21       nodule   NN
22     province   NN
23          key   JJ
24   components  NNS
25  polychaetes  NNS,        word  POS
0      orms  NNS
1  violence   NN
2     women  NNS
3     girls  NNS
4   defined  VBN,           word  POS
3     document   NN
4     provides  VBZ
5   additional   JJ
6  information   NN
7   comparison   NN,         word  POS
3   strategy   NN
4      focus   VB
5     solely   RB
6  promoting  VBG
7       cost   NN,          word  POS
1     forward   RB
2     address   VB
3  challenges  NNS
4     consult   VB
5     friends  NNS,            word  POS
1      possible   JJ
2  consequences  NNS
3           use   NN
4       nuclear   JJ
5       weapons  NNS,       word  POS
0       un  NNP
1  re

## Part 3 - convert the samples into a Pandas DataFrame (10 points)

Here, you will take the samples and create a table whose columns are the features and the class and whose rows are the samples.  All the features and the class will be binary.  Note that there may be many columns, in the hundreds or thousands depending on the diversity of the final two consonants of the non-stop-words in the dataset, but the sum of all rows will be five.

In [6]:
fulldf = mc.create_df(all_samples)
fulldf[25000:25010]

,"(1, ab)","(1, ac)","(1, ad)","(1, ae)","(1, af)","(1, ag)","(1, ah)","(1, ai)","(1, ak)","(1, al)",...,"(4, yi)","(4, ym)","(4, yn)","(4, yp)","(4, ys)","(4, yz)","(4, z)","(4, za)","(4, ze)",target
25000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Part 4 - extract training and testing sets (3 points)

Here, you will create the training and testing datasets in order to train the model.  This will be based on a test percentage.  Round up if the percentage does not divide evenly into the sample size.  You will need to separate the class column into the y-values for the classifier.

In [7]:
train_X, train_y, test_X, test_y = mc.split_samples(fulldf, test_percent=20)
len(train_X), len(train_y), len(test_X), len(test_y)

(40000, 40000, 10000, 10000)

## Part 5 - train models (3 points)

You will then train and return two support vector machine (SVM) models using the sklearn SVC class.  You should allow a choice between linear and radial basis function kernels.

In [8]:
model_linear = mc.train(train_X, train_y, kernel='linear')
model_rbf = mc.train(train_X, train_y, kernel="rbf")
model_linear, model_rbf

(SVC(kernel='linear'), SVC())

## Part 6 - evaluate the models (5 points)

You will calculate and print precision, recall, and F-measure for the models on the test data. In `notes.md`, write down your comparison of these simple measures on the two models and any thoughts you might have on what they mean. (It could be very short, and since the samples do not stay stable between runs, you can save the evaluation scores in `notes.md` too.)

In [9]:

mc.eval_model(model_rbf, test_X, test_y)

(array([0.72556126, 0.79104478]),
 array([0.99806121, 0.01907161]),
 array([0.84027049, 0.03724526]))

In [10]:
mc.eval_model(model_linear, test_X, test_y)

(array([0.7238269 , 0.40601504]),
 array([0.98905969, 0.01943145]),
 array([0.83590824, 0.03708791]))

## Part Bonus - try another sort of model from sklearn (5 points)

Write a separate, command-line script (not a notebook) uses `mycode.py` to do all of the above, except that it trains a non-SVM classifier model.  Any non-trivial model available in sklearn will do. Explain how to run your code and the results of your own evaluation in `notes.md`, including any observations or opinions you may have on the classifier method you used in comparison to SVM.

## Submission

Push to your fork of the GitHub repository (which must be made public) and submit the URL of your repository in Canvas.  You can submit this notebook with the output from your run, as long as you do not modify the code or text in it without permission from us.  

In [13]:
mc.eval_Naive_Bayes(train_X, train_y, test_X, test_y)

(array([0.82564103, 0.28210198]),
 array([0.04459216, 0.97553077]),
 array([0.08461437, 0.4376463 ]))